In [0]:
-- Drop and recreate the performance test log table with log_id
DROP TABLE IF EXISTS gap_catalog.log.performance_test_log;
CREATE TABLE IF NOT EXISTS gap_catalog.log.performance_test_log (
    log_id STRING NOT NULL DEFAULT UUID() COMMENT 'Unique identifier for each log entry',
    test_case_name STRING NOT NULL COMMENT 'Name of the test case being executed',
    step_name STRING COMMENT 'Name of the specific step within the test case',
    start_time TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP() COMMENT 'Start time of the test execution',
    end_time TIMESTAMP COMMENT 'End time of the test execution',
    source_sys_origin STRING COMMENT 'Source system origin of the data being processed',
    started_by STRING NOT NULL COMMENT 'User who initiated the test',
    status STRING NOT NULL COMMENT 'Status of the test (RUNNING, COMPLETED, FAILED, etc.)',
    process_key STRING COMMENT 'Unique identifier for the test process',
    p_load_date DATE COMMENT 'Load date parameter for the test execution',
    target_table STRING COMMENT 'Name of the target table being processed',
    target_insert_count BIGINT COMMENT 'Number of records inserted into target table',
    target_update_count BIGINT COMMENT 'Number of records updated in target table',
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP() COMMENT 'Record creation timestamp',
    updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP() COMMENT 'Record last update timestamp'
)
USING DELTA
TBLPROPERTIES (
    'delta.feature.allowColumnDefaults' = 'supported',
    'delta.autoOptimize.optimizeWrite' = 'true',
    'delta.autoOptimize.autoCompact' = 'true'
)
COMMENT 'Performance testing log table to track test executions with detailed step-level metrics and target table processing counts'
;

-- Updated procedure to start a performance test (returns log_id)
CREATE OR REPLACE PROCEDURE gap_catalog.log.start_performance_test(
    IN test_case_name STRING,
    IN step_name STRING,
    IN psource_sys_origin STRING,
    IN process_key STRING,
    IN target_table STRING,
    OUT log_id STRING,
    IN p_load_date STRING DEFAULT NULL
)
LANGUAGE SQL
SQL SECURITY INVOKER
COMMENT 'Procedure to log the start of a performance test with step details and return log_id'
AS
BEGIN
    SET log_id = UUID();
    
    INSERT INTO gap_catalog.log.performance_test_log (
        log_id,
        test_case_name,
        step_name,
        start_time,
        source_sys_origin,
        started_by,
        status,
        process_key,
        target_table,
        p_load_date,
        created_at,
        updated_at
    )
    VALUES (
        log_id,
        test_case_name,
        step_name,
        CURRENT_TIMESTAMP(),
        psource_sys_origin,
        CURRENT_USER(),
        'RUNNING',
        process_key,
        target_table,
        DATE(p_load_date),
        CURRENT_TIMESTAMP(),
        CURRENT_TIMESTAMP()
    );

    SELECT CONCAT('Performance test started: ', test_case_name, 
                  ' - Step: ', COALESCE(step_name, 'N/A'),
                  ' - Process key: ', process_key,
                  ' - Target: ', target_table,
                  ' - Log ID: ', log_id) as message, log_id as returned_log_id;
END;

-- Updated procedure to complete a performance test (using log_id, always COMPLETED status)
CREATE OR REPLACE PROCEDURE gap_catalog.log.complete_performance_test(
    IN log_id STRING
)
LANGUAGE SQL
SQL SECURITY INVOKER
COMMENT 'Procedure to log the completion of a performance test'
AS
BEGIN
    UPDATE gap_catalog.log.performance_test_log
    SET 
        end_time = CURRENT_TIMESTAMP(),
        status = 'COMPLETED',
        updated_at = CURRENT_TIMESTAMP()
    WHERE log_id = log_id;
    
    SELECT CONCAT('Performance test completed with status: COMPLETED', 
                  ' for log ID: ', log_id) as message;
END;

-- Updated procedure to update target counts with additional parameters
CREATE OR REPLACE PROCEDURE gap_catalog.log.update_target_counts(
    IN process_key STRING,
    IN target_table STRING,
    IN psource_sys_origin STRING,
    IN ptarget_insert_count BIGINT,
    IN ptarget_update_count BIGINT
)
LANGUAGE SQL
SQL SECURITY INVOKER
COMMENT 'Procedure to update target table processing counts for a specific process key, target table, and source system'
AS
BEGIN
    UPDATE gap_catalog.log.performance_test_log
    SET 
        target_insert_count = ptarget_insert_count,
        target_update_count = ptarget_update_count,
        updated_at = CURRENT_TIMESTAMP()
    WHERE process_key = process_key
      AND target_table = target_table
      AND source_sys_origin = psource_sys_origin;
    
    SELECT CONCAT('Target counts updated for process key: ', process_key,
                  ', target table: ', target_table,
                  ', source system: ', psource_sys_origin,
                  ' - Inserts: ', ptarget_insert_count,
                  ', Updates: ', ptarget_update_count) as message;
END;

-- Select all records from the performance test log
SELECT * FROM gap_catalog.log.performance_test_log order by start_time desc;
DELETE FROM gap_catalog.log.performance_test_log ;
/*
-- Updated example usage:

-- 1. Start a performance test and capture the log_id
DECLARE my_log_id STRING;

CALL gap_catalog.log.start_performance_test(
    test_case_name => 'ADS_RDS_Q',
    step_name => 'Extract_Data',
    psource_sys_origin  => 'RDS_MEPOPERATIONTYPE',
    process_key => '889',
    target_table => 'ads_rds_event_types_mep',
    p_load_date => '2025-09-24',
    log_id => my_log_id
);

-- 2. Complete the performance test using the captured log_id
CALL gap_catalog.log.complete_performance_test(
    log_id => my_log_id
);

-- 3. Update target counts with additional parameters
CALL gap_catalog.log.update_target_counts(
    process_key => '889',
    target_table => 'ads_rds_event_types_mep',
    psource_sys_origin => 'RDS_MEPOPERATIONTYPE',
    ptarget_insert_count => 50000,
    ptarget_update_count => 5000
);
*/

-- Query to view performance test logs with log_id
SELECT 
    log_id,
    test_case_name,
    step_name,
    start_time,
    end_time,
    CASE 
        WHEN end_time IS NOT NULL THEN 
            CAST((UNIX_TIMESTAMP(end_time) - UNIX_TIMESTAMP(start_time)) AS STRING) || ' seconds'
        ELSE 'Still running'
    END as duration,
    source_sys_origin,
    started_by,
    status,
    process_key,
    p_load_date,
    target_table,
    target_insert_count,
    target_update_count,
    CASE 
        WHEN target_insert_count IS NOT NULL AND target_update_count IS NOT NULL 
        THEN target_insert_count + target_update_count 
        ELSE NULL 
    END as total_target_records,
    created_at
FROM gap_catalog.log.performance_test_log
ORDER BY created_at DESC;